<a href="https://colab.research.google.com/github/FBehrad/AI-in-medical-field/blob/main/How_to_load_and_visualize_medical_images_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Install required packages

In [ ]:
!pip install SimpleITK 
!pip install nibabel

     |████████████████████████████████| 48.4 MB 5.3 kB/s 


3. Import reqired libraries

In [ ]:
import zipfile  # For file extraction
import glob  # For retrieving files/pathnames matching a specified pattern
import re # specifies a set of strings that matches it
import SimpleITK as sitk
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

4. Extract datasets

In [ ]:
def extract_dataset(addr,folder_name):
  dataset_path = addr 
  zfile = zipfile.ZipFile(dataset_path)
  zfile.extractall(folder_name)

In [ ]:
extract_dataset('/content/drive/MyDrive/MICCAI_BraTS_2018_Data_Training.zip','Training') # Replace with your dataset path

In [ ]:
t1 = glob.glob('/content/Training/*GG/*/*t1.nii.gz')
t2 = glob.glob('/content/Training/*GG/*/*t2.nii.gz')
flair = glob.glob('/content/Training/*GG/*/*flair.nii.gz')
t1ce = glob.glob('/content/Training/*GG/*/*t1ce.nii.gz')
seg = glob.glob('/content/Training/*GG/*/*seg.nii.gz')  # Ground Truth
pattern = re.compile('/content/Training/.*_(\w*)\.nii\.gz')

data_paths = [{
    pattern.findall(item)[0]:item for item in items
}
for items in list(zip(t1, t2, t1ce, flair, seg))]

print('number of training examples' ,len(data_paths))
print(data_paths[0])

number of training examples 285
{'t1': '/content/Training/LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_t1.nii.gz', 't2': '/content/Training/LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_t2.nii.gz', 't1ce': '/content/Training/LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_t1ce.nii.gz', 'flair': '/content/Training/LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_flair.nii.gz', 'seg': '/content/Training/LGG/Brats18_TCIA10_442_1/Brats18_TCIA10_442_1_seg.nii.gz'}


5. Read images

In [ ]:
def read_img_sitk(img_path):
  image_data = sitk.ReadImage(img_path)
  return image_data 

In [ ]:
def read_img_nii(img_path):
  image_data = np.array(nib.load(img_path).get_fdata())
  return image_data  

In [ ]:
np_img = read_img_nii(data_paths[0]['flair'])
sitk_img = read_img_sitk(data_paths[0]['flair'])

In [ ]:
#Check shape of images

np_shape = np_img.shape
sitk_shape = sitk_img.GetSize()
print("Shape of np_img : ", np_shape)
print("Shape of sitk_img : ", sitk_shape)

Shape of np_img :  (240, 240, 155)
Shape of sitk_img :  (240, 240, 155)


6. Conversion between numpy and SimpleITK

In [ ]:
sitk_img2 = sitk.GetImageFromArray(np_img)

In [ ]:
sitk_img2.GetSize()

(155, 240, 240)

In [ ]:
np_img2 = sitk.GetArrayFromImage(sitk_img2)

In [ ]:
np_img2.shape

(240, 240, 155)

7. Visualize samples

In [ ]:
@interact
def explore_3dimage(layer = (0,154) , modality=['t1', 't2', 't1ce', 'flair'] , view = ['axial' , 'sagittal' , 'coronal'], patient= (0,284) ): 
    if modality == 't1':
      modal = 't1'
    elif modality == 't2':
      modal = 't2'
    elif modality == 't1ce':
      modal = 't1ce'
    elif modality == 'flair':
      modal = 'flair'
    else :
      print("Error")

    
    image = read_img_sitk(data_paths[patient][modal])
    array_view = sitk.GetArrayViewFromImage(image)

    if view == 'axial':
      array_view = array_view[layer, :,:]
    elif view == 'coronal':
      array_view = array_view[:, layer, :]
    elif view == 'sagittal':
      array_view = array_view[:, : ,layer]
    else:
      print("Error")

    plt.figure(figsize=(10, 5))
    plt.imshow(array_view, cmap='gray')
    plt.title('Explore Layers of Brain', fontsize=10)
    plt.axis('off')

interactive(children=(IntSlider(value=77, description='layer', max=154), Dropdown(description='modality', opti…

8. Write images

In [ ]:
final_paths = '/content/example.nii.gz'
sitk.WriteImage(sitk_img2,final_paths)